In [ ]:
import numpy as np
import pandas as pd
import requests
import gzip
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import methylcheck
from scipy.cluster.hierarchy import fcluster
from pyensembl import EnsemblRelease
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MinMaxScaler, RobustScaler
from sklearn import metrics 
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_selection import SelectFromModel, VarianceThreshold, RFE, SelectKBest, chi2, f_classif
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import Lasso, Ridge , LinearRegression
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBRegressor, plot_importance
import joblib

# Exploratory Data Analysis

In [ ]:
# Load data
pd.set_option('display.max_columns', None)

covariates = pd.read_csv('data/CLL_Covariates.txt', sep='\t')
drugs = pd.read_csv('data/CLL_Drugs.txt', sep='\t')
methylation = pd.read_csv('data/CLL_Methylation.txt', sep='\t')
mutations = pd.read_csv('data/CLL_Mutations.txt', sep='\t')
mrna = pd.read_csv('data/CLL_mRNA.txt', sep='\t')

In [ ]:
# Explore
for df_name, df in [("covariates", covariates), ("drugs", drugs), ("methylation", methylation), ("mutations", mutations), ("mrna", mrna)]:
    print(f"\n\nDataFrame: {df_name}\n")
    df.info()

## Visualizations

### covariates

In [ ]:
# Gender distribution
sns.set(style="whitegrid")

plt.figure(figsize=(12, 7))
palette = sns.color_palette('Set2', n_colors=len(covariates['Gender'].unique()))

ax = sns.countplot(data=covariates, x='Gender', palette=palette, edgecolor='black')
ax.set_xticklabels(['Male', 'Female'], fontsize=12, rotation=45)

plt.title('Distribution of Gender in the Cohort', fontsize=18)
plt.xlabel('Gender', fontsize=14)
plt.ylabel('Count', fontsize=14)

for p in ax.patches:
    plt.text(
        p.get_x() + p.get_width() / 2,
        p.get_height() + 2,
        f'{int(p.get_height())}',
        ha='center',
        fontsize=12
    )

plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### drugs

In [ ]:
# Heatmap with clustering for drugs and samples
g = sns.clustermap(
    drugs.fillna(0), 
    cmap="YlGnBu", 
    annot=False,  
    linewidths=0.5, 
    linecolor='grey', 
    cbar_kws={'label': 'Response Level'}, 
    method='average', 
    metric='euclidean', 
    figsize=(20, 15)
)

g.ax_heatmap.set_title('Drug Responses Across Samples', fontsize=20, fontweight='bold')
g.ax_heatmap.set_xlabel('Samples', fontsize=16)
g.ax_heatmap.set_ylabel('Drugs', fontsize=16)

row_threshold = 5  
g.ax_row_dendrogram.axvline(x=row_threshold, color='r', linestyle='--', linewidth=1)

col_threshold = 5  
g.ax_col_dendrogram.axhline(y=col_threshold, color='r', linestyle='--', linewidth=1)

colorbar = g.cax
colorbar.yaxis.set_ticks_position('left')
colorbar.yaxis.set_label_position('left')

plt.setp(g.ax_heatmap.get_xticklabels(), rotation=90)
plt.tight_layout()
plt.show()

### methylation

In [ ]:
# Methylation distribution
methylation_flattened = methylation.values.flatten()[~np.isnan(methylation.values.flatten())]

plt.figure(figsize=(14, 8))
sns.histplot(methylation_flattened, kde=True, bins=50, color='skyblue') 
plt.title('Methylation Level Distribution Across Samples', fontsize=20, fontweight='bold')
plt.xlabel('Methylation Level', fontsize=16, fontweight='bold')
plt.ylabel('Frequency', fontsize=16, fontweight='bold')
plt.grid(True, linestyle='--', alpha=0.7)  
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

### mutations

In [ ]:
# Convert mutation values to integers
mutations = mutations.astype('float').astype('Int64')

In [ ]:
# Heatmap of mutations
mutations_imputed = mutations.fillna(0).astype(int)

clustermap = sns.clustermap(
    mutations_imputed,
    cmap="coolwarm",  
    linewidths=0.5, 
    linecolor='grey', 
    cbar_kws={'label': 'Mutation Presence'}, 
    method='average', 
    metric='euclidean',  
    figsize=(20, 15),
    xticklabels=True,
    yticklabels=True
)

clustermap.ax_heatmap.set_title('Mutation Presence Across Samples', fontsize=20, fontweight='bold')
clustermap.ax_heatmap.set_xlabel('Patients', fontsize=16)
clustermap.ax_heatmap.set_ylabel('Mutations', fontsize=16)

plt.setp(clustermap.ax_heatmap.get_xticklabels(), rotation=90)

row_threshold = 5  
clustermap.ax_row_dendrogram.axvline(x=row_threshold, color='r', linestyle='--', linewidth=1)

clustermap.cax.yaxis.set_label_position('left')
clustermap.cax.yaxis.set_ticks_position('left')

plt.tight_layout()
plt.show()

# Pre-processing

## Map Eensembl_ID / Gene_names

In [ ]:
# Initialize EnsemblRelease
ensrel = EnsemblRelease(109)

In [ ]:
# Create column with GeneIDs
mrna['GeneID'] = mrna.index

In [ ]:
# Function to map Ensembl IDs to gene names
def map_ensembl_to_gene_name(ensembl_id):
    try:
        gene = ensrel.gene_by_id(ensembl_id)
        return gene.gene_name
    except ValueError:
        return None

In [ ]:
# Map GeneIDs to GeneNames
mrna['GeneName'] = mrna['GeneID'].apply(map_ensembl_to_gene_name)
mrna

In [ ]:
# Filter empty rows
filtered_mRNA = mrna[mrna['GeneName'].isna() | (mrna['GeneName'] == '')]

print("\nRows with empty or None in 'GeneName' column:")
filtered_mRNA

In [ ]:
# Check the missing genes in Ensembl
missing_gene_ids = mrna[mrna['GeneName'].isna() | (mrna['GeneName'] == '')]['GeneID'].unique()

def check_gene_names(gene_ids):
    url = 'https://rest.ensembl.org/lookup/id/{}?content-type=application/json'
    missing_names = {}

    for gene_id in gene_ids:
        response = requests.get(url.format(gene_id))
        if response.status_code == 200:
            data = response.json()
            if 'display_name' in data:
                missing_names[gene_id] = data['display_name']
            else:
                missing_names[gene_id] = 'No GeneName available'
        else:
            missing_names[gene_id] = 'Request failed'

    return missing_names

missing_gene_names = check_gene_names(missing_gene_ids)

for gene_id, name in missing_gene_names.items():
    print(f"GeneID: {gene_id}, GeneName: {name}")

In [ ]:
# Fill empty strings with GeneID
mrna['GeneName'] = mrna['GeneName'].replace('', pd.NA).fillna(mrna['GeneID'])

In [ ]:
# Check again for missing gene names
filtered_mRNA = mrna[mrna['GeneName'].isna() | (mrna['GeneName'] == '')]

print("\nRows with empty or None in 'GeneName' column:")
filtered_mRNA

In [ ]:
# Clean the expression data
expr = mrna.set_index('GeneName').drop(columns=['GeneID']).rename_axis(None)
expr

## Duplicates

In [ ]:
# Check if indices are unique
if covariates.index.is_unique:
    print("Indices in 'covariates' are unique.")
else:
    print("Indices in 'covariates' are not unique. Duplicates:")
    print(covariates.index[covariates.index.duplicated()].tolist())

In [ ]:
# Function to check for duplicates
def check_duplicates(df, name):
    duplicate_rows = df[df.duplicated()]
    if not duplicate_rows.empty:
        print(f"Dataset '{name}' has {duplicate_rows.shape[0]} duplicate rows at indices:")
        print(duplicate_rows.index.tolist())
    else:
        print(f"Dataset '{name}' has no duplicate rows.")

In [ ]:
# Check for duplicates
check_duplicates(drugs, 'drugs')
check_duplicates(methylation, 'methylation')
check_duplicates(mutations, 'mutations')
check_duplicates(expr, 'expr')

## Missing data

In [ ]:
# Function to check missing data
def check_missing_data(datasets):
    for name, dataset in datasets.items():
        missing_count = dataset.isnull().sum().sum()
        print(f"Dataset '{name}' has {missing_count} missing values.")

In [ ]:
# Function to count missing data
def count_missing_data(data):
    missing_val = data.isnull().sum()
    cols_with_missing_val = missing_val[missing_val > 0]

    data_missing = pd.DataFrame([cols_with_missing_val.index, cols_with_missing_val.values])
    data_missing.index = ['Sample', 'Missing Values Count']

    print(f"Missing columns total: {len(cols_with_missing_val)}")
    return data_missing

In [ ]:
# Function to get row statistics
def calc_row_stats(data):
    # Calculate statistics per row
    row_stats = data.agg(['min', 'max', 'mean'], axis=1, skipna=True).rename(columns={'min': 'Min', 'max': 'Max', 'mean': 'Mean'})

    # Print row statistics
    print("Row Statistics:")
    # print(row_stats)

    return row_stats

In [ ]:
# Function to check zeros, inf, and negative values
def check_special_values(data):    
    zeros_count_total = (data == 0).sum().sum()
    infinite_values = data.isin([float('inf'), float('-inf')]).sum().sum()
    negative_values = (data < 0).sum().sum()

    print(f"Number of zeros: {zeros_count_total}")
    print(f"Number of infinite values: {infinite_values}")
    print(f"Number of negative values: {negative_values}")

    return {
        'zeros_count_total': zeros_count_total,
        'infinite_values': infinite_values,
        'negative_values': negative_values
    }

In [ ]:
datasets = {
    'covariates': covariates,
    'drugs': drugs,
    'methylation': methylation,
    'mutations': mutations,
    'expr': expr
}

check_missing_data(datasets)

### drugs

In [ ]:
drugs

In [ ]:
# Count missing data
data_drugs = count_missing_data(drugs)
data_drugs

In [ ]:
# Row statistics
drugs_row_stats = calc_row_stats(drugs)
drugs_row_stats

In [ ]:
# Check for 0, inf and negative values
special_values = check_special_values(drugs)

In [ ]:
# Histogram of drug distribution
all_values = drugs.values.flatten()[np.isfinite(drugs.values.flatten())]

plt.figure(figsize=(12, 8))
n, bins, patches = plt.hist(all_values, bins=100, color='skyblue', edgecolor='black')

plt.title('Drug Response Distribution Across Samples', fontsize=16)
plt.xlabel('Drug Response', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(axis='y', alpha=0.75)

mean_value = np.mean(all_values)
median_value = np.median(all_values)
plt.axvline(mean_value, color='red', linestyle='dashed', linewidth=2, label=f'Mean: {mean_value:.2f}')
plt.axvline(median_value, color='green', linestyle='dashed', linewidth=2, label=f'Median: {median_value:.2f}')
plt.legend(fontsize=12)
plt.text(mean_value, plt.ylim()[1]*0.9, f'Mean: {mean_value:.2f}', color='red', fontsize=12, ha='left')
plt.text(median_value, plt.ylim()[1]*0.8, f'Median: {median_value:.2f}', color='green', fontsize=12, ha='right')
plt.show()

In [ ]:
# Fill NaN values with the median 
drugs_imputed = drugs.apply(lambda row: row.fillna(row.median()), axis=1)

### methylation

In [ ]:
methylation

In [ ]:
# Count missing data 
data_meth = count_missing_data(methylation)
data_meth

In [ ]:
# Row statistics
meth_row_stats = calc_row_stats(methylation)
meth_row_stats

In [ ]:
# Check for 0, inf and negative values
special_values = check_special_values(methylation)

In [ ]:
# Histogram of methylation distribution
all_values = methylation.values.flatten()[np.isfinite(methylation.values.flatten())]

plt.figure(figsize=(12, 8))
n, bins, patches = plt.hist(all_values, bins=100, color='skyblue', edgecolor='black')

plt.title('Methylation Distribution Across Samples', fontsize=16)
plt.xlabel('Methylation', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(axis='y', alpha=0.75)

mean_value = np.mean(all_values)
median_value = np.median(all_values)
plt.axvline(mean_value, color='red', linestyle='dashed', linewidth=2, label=f'Mean: {mean_value:.2f}')
plt.axvline(median_value, color='green', linestyle='dashed', linewidth=2, label=f'Median: {median_value:.2f}')
plt.legend(fontsize=12)
plt.text(mean_value, plt.ylim()[1]*0.9, f'Mean: {mean_value:.2f}', color='red', fontsize=12, ha='left')
plt.text(median_value, plt.ylim()[1]*0.8, f'Median: {median_value:.2f}', color='green', fontsize=12, ha='right')
plt.show()

In [ ]:
# Fill NaN values with the median
methylation_imputed = methylation.apply(lambda row: row.fillna(row.median()), axis=1)

### mutations

In [ ]:
mutations

In [ ]:
# Count missing data 
data_mut = count_missing_data(mutations)
data_mut

In [ ]:
# Check for 0, inf and negative values
special_values = check_special_values(mutations)

In [ ]:
# Mode imputation 
mutations_imputed = mutations.apply(lambda row: row.fillna(row.mode().iloc[0] if not row.mode().empty else np.nan), axis=1)

In [ ]:
# Check before and after imputation
counts_before = {"0": (mutations == 0).sum().sum(), "1": (mutations == 1).sum().sum(), "NaN": mutations.isnull().sum().sum()}
counts_after = {"0": (mutations_imputed == 0).sum().sum(), "1": (mutations_imputed == 1).sum().sum(), "NaN": mutations_imputed.isnull().sum().sum()}

counts_df = pd.DataFrame([counts_before, counts_after], index=["Before Imputation", "After Imputation"])
counts_df

In [ ]:
# Identify the imputed data with ones
new_ones = (mutations.isnull()) & (mutations_imputed == 1)
rows_with_new_ones = mutations_imputed[new_ones.any(axis=1)]
rows_with_new_ones

### mrna

In [ ]:
expr

In [ ]:
# Count missing data 
data_expr = count_missing_data(expr)
data_expr

In [ ]:
# Check for common samples between data
common_samples = set(data_drugs.index).intersection(data_meth.index, data_expr.index) - {'Sample', 'Missing Values Count'}

# Print the result
print(f"Common Samples: {len(common_samples)}" if common_samples else "No common samples found after filtering.")
print(common_samples)

In [ ]:
# Row statistics
expr_row_stats = calc_row_stats(expr)
expr_row_stats

In [ ]:
# Check for 0, inf and negative values
special_values = check_special_values(expr)

#### Filtering

In [ ]:
# Remove low-expression genes
threshold = 0.5 

filtered_expr = expr.loc[expr.mean(axis=1) >= threshold]
filtered_expr

In [ ]:
# Quantiles of variance per gene
quantiles = filtered_expr.var(axis=1).quantile([0.25, 0.5, 0.75, 0.9])

print("25th percentile:", quantiles[0.25])
print("50th percentile (median):", quantiles[0.5])
print("75th percentile:", quantiles[0.75])
print("90th percentile:", quantiles[0.9])

In [ ]:
# Remove genes with low variance
variance_threshold = 0.5

filtered_expr_2 = filtered_expr.loc[filtered_expr.var(axis=1) >= variance_threshold]
filtered_expr_2

In [ ]:
# Quantiles of variance per sample
quantiles = filtered_expr_2.var(axis=0).quantile([0.25, 0.5, 0.75, 0.9])

print("25th percentile:", quantiles[0.25])
print("50th percentile (median):", quantiles[0.5])
print("75th percentile:", quantiles[0.75])
print("90th percentile:", quantiles[0.9])

In [ ]:
# Histogram of expression distribution
all_values = filtered_expr_2.values.flatten()[np.isfinite(filtered_expr_2.values.flatten())]

plt.figure(figsize=(12, 8))
n, bins, patches = plt.hist(all_values, bins=100, color='skyblue', edgecolor='black')

plt.title('Distribution of Expression Across Samples', fontsize=16)
plt.xlabel('Expression Level', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.grid(axis='y', alpha=0.75)

mean_value = np.mean(all_values)
median_value = np.median(all_values)
plt.axvline(mean_value, color='red', linestyle='dashed', linewidth=2, label=f'Mean: {mean_value:.2f}')
plt.axvline(median_value, color='green', linestyle='dashed', linewidth=2, label=f'Median: {median_value:.2f}')
plt.legend(fontsize=12)
plt.text(mean_value, plt.ylim()[1]*0.9, f'Mean: {mean_value:.2f}', color='red', fontsize=12, ha='left')
plt.text(median_value, plt.ylim()[1]*0.8, f'Median: {median_value:.2f}', color='green', fontsize=12, ha='right')
plt.show()

In [ ]:
# Fill NaN values with the median 
expr_imputed = filtered_expr_2.apply(lambda row: row.fillna(row.median()), axis=1)

In [ ]:
# Check for missing data
datasets = {
    'covariates': covariates,
    'drugs': drugs_imputed,
    'methylation': methylation_imputed,
    'mutations': mutations_imputed,
    'expr': expr_imputed
}

check_missing_data(datasets)

## Normalization

### drugs_imputed

In [ ]:
# Z-score normalization
scaler = StandardScaler()

drugs_normalized = scaler.fit_transform(drugs_imputed)
drugs_normalized = pd.DataFrame(drugs_normalized, index=drugs_imputed.index, columns=drugs_imputed.columns)
drugs_normalized.describe()

In [ ]:
# Check normalized data
print(drugs_normalized.describe().T[['mean', 'std']])

Mean: The mean values for each sample are very close to zero, which is what you would expect from normalized data.
Standard Deviation: The standard deviation values for each sample are very close to 1.001617. Ideally, we expect a standard deviation of 1, but the slight deviation is due to numerical precision and rounding errors, which are acceptable.

In [ ]:
# Normalized drug response distribution
subset_drugs_normalized = drugs_normalized.iloc[:, :5]

sns.pairplot(subset_drugs_normalized)
plt.suptitle('Pairplot of Normalized Drug Data', y=1.02)
plt.show()

In [ ]:
# Distribution before and after normalization
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# Before normalization
sns.histplot(drugs_imputed.values.flatten(), bins=100, ax=axs[0])
axs[0].set_title('Histogram of Drug Response Values (Before Normalization)')
axs[0].set_xlabel('Drug Response')
axs[0].set_ylabel('Frequency')

# After normalization
sns.histplot(drugs_normalized.values.flatten(), bins=100, ax=axs[1])
axs[1].set_title('Histogram of Z-score Normalized Drug Response Values')
axs[1].set_xlabel('Drug Response')
axs[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Z-score normalization
drugs_zscore = scaler.fit_transform(drugs_imputed)
drugs_zscore_df = pd.DataFrame(drugs_zscore, index=drugs_imputed.index, columns=drugs_imputed.columns)

# Min-Max scaling
minmax_scaler = MinMaxScaler()
drugs_minmax = minmax_scaler.fit_transform(drugs_imputed)
drugs_minmax_df = pd.DataFrame(drugs_minmax, index=drugs_imputed.index, columns=drugs_imputed.columns)

# Robust scaling
robust_scaler = RobustScaler()
drugs_robust = robust_scaler.fit_transform(drugs_imputed)
drugs_robust_df = pd.DataFrame(drugs_robust, index=drugs_imputed.index, columns=drugs_imputed.columns)

# Log transformation
drugs_log = np.log1p(drugs_imputed)
drugs_log_df = pd.DataFrame(drugs_log, index=drugs_imputed.index, columns=drugs_imputed.columns)

In [ ]:
# Comparison between different normalization techniques
fig = plt.figure(figsize=(10, 10))

# Before normalization
ax_main = fig.add_subplot(3, 1, 1)
sns.histplot(drugs_imputed.values.flatten(), bins=100, ax=ax_main, kde=True)
ax_main.set_title('Not Normalized Data')
ax_main.set_xlabel('Value')
ax_main.set_ylabel('Frequency')

# Z-score
ax1 = fig.add_subplot(3, 2, 3)
sns.histplot(drugs_zscore_df.values.flatten(), bins=100, ax=ax1, kde=True)
ax1.set_title('Z-score Normalization')
ax1.set_xlabel('Value')
ax1.set_ylabel('Frequency')

# Min-Max
ax2 = fig.add_subplot(3, 2, 4)
sns.histplot(drugs_minmax_df.values.flatten(), bins=100, ax=ax2, kde=True)
ax2.set_title('Min-Max Scaling')
ax2.set_xlabel('Value')
ax2.set_ylabel('Frequency')

# Robust
ax3 = fig.add_subplot(3, 2, 5)
sns.histplot(drugs_robust_df.values.flatten(), bins=100, ax=ax3, kde=True)
ax3.set_title('Robust Scaling')
ax3.set_xlabel('Value')
ax3.set_ylabel('Frequency')

# Log
ax4 = fig.add_subplot(3, 2, 6)
sns.histplot(drugs_log_df.values.flatten(), bins=100, ax=ax4, kde=True)
ax4.set_title('Log Transformation')
ax4.set_xlabel('Value')
ax4.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
drugs_normalized = drugs_robust_df

### methylation_imputed

In [ ]:
# Quantile normalization
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
methylation_normalized = pd.DataFrame(quantile_transformer.fit_transform(methylation_imputed), 
                                               index=methylation_imputed.index, 
                                               columns=methylation_imputed.columns)

In [ ]:
# Distribution before and after normalization
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# Before normalization
sns.histplot(methylation_imputed.values.flatten(), bins=100, ax=axs[0])
axs[0].set_title('Methylation (Before Normalization)')
axs[0].set_xlabel('Methylation Level')
axs[0].set_ylabel('Frequency')

# After normalization
sns.histplot(methylation_normalized.values.flatten(), bins=100, ax=axs[1])
axs[1].set_title('Normalized Methylation')
axs[1].set_xlabel('Methylation Level')
axs[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Boxplot and QQplot for methylation
data_flattened = methylation_normalized.values.flatten()

# Create the figure and subplots
fig, ax = plt.subplots(1, 2, figsize=(16, 8))

# Boxplot
sns.boxplot(data=data_flattened, ax=ax[0], color='skyblue')
ax[0].set_title('Boxplot of Methylation Values')
ax[0].set_xlabel('Methylation Level')

# QQ plot
stats.probplot(data_flattened, dist="norm", plot=ax[1])
ax[1].set_title('QQ Plot of Methylation Values')
ax[1].set_xlabel('Theoretical Quantiles')
ax[1].set_ylabel('Sample Quantiles')

plt.tight_layout()
plt.show()

In [ ]:
# Save to CSV file
csv_file_path = 'data/methylation_normalized.csv'
methylation_normalized.to_csv(csv_file_path)

### expr_imputed

In [ ]:
# Log Transformation
expr_log = np.log2(expr_imputed)

In [ ]:
# Plot log-transformed  expr data distribution
all_values_log = expr_log.values.flatten()

plt.figure(figsize=(10, 6))
plt.hist(all_values_log, bins=100, color='skyblue', edgecolor='black')
plt.title('Histogram of Log-Transformed Gene Expression Values')
plt.xlabel('Log2(Expression Level)')
plt.ylabel('Frequency')
plt.axvline(np.mean(all_values_log), color='r', linestyle='--', linewidth=1.5, label=f'Mean: {np.mean(all_values_log):.2f}')
plt.axvline(np.median(all_values_log), color='g', linestyle='--', linewidth=1.5, label=f'Median: {np.median(all_values_log):.2f}')
plt.legend()
plt.show()

In [ ]:
# Z-score normalization
expr_zscore = scaler.fit_transform(expr_imputed)
expr_zscore_df = pd.DataFrame(expr_zscore, index=expr_imputed.index, columns=expr_imputed.columns)

# Min-Max scaling
minmax_scaler = MinMaxScaler()
expr_minmax = minmax_scaler.fit_transform(expr_imputed)
expr_minmax_df = pd.DataFrame(expr_minmax, index=expr_imputed.index, columns=expr_imputed.columns)

# Robust scaling
robust_scaler = RobustScaler()
expr_robust = robust_scaler.fit_transform(expr_imputed)
expr_robust_df = pd.DataFrame(expr_robust, index=expr_imputed.index, columns=expr_imputed.columns)

# Log transformation
expr_log = np.log1p(expr_imputed)
expr_log_df = pd.DataFrame(expr_log, index=expr_imputed.index, columns=expr_imputed.columns)

# Quantile transformation
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
expr_quantile_normalized = quantile_transformer.fit_transform(expr_imputed)
expr_quantile_normalized_df = pd.DataFrame(expr_quantile_normalized, index=expr_imputed.index, columns=expr_imputed.columns)

In [ ]:
# Comparison between different normalization techniques
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Before normalization
sns.histplot(expr_imputed.values.flatten(), bins=100, ax=axes[0, 0], kde=True)
axes[0, 0].set_title('Not Normalized Data')
axes[0, 0].set_xlabel('Value')
axes[0, 0].set_ylabel('Frequency')

# Z-score
sns.histplot(expr_zscore_df.values.flatten(), bins=100, ax=axes[0, 1], kde=True)
axes[0, 1].set_title('Z-score Normalization')
axes[0, 1].set_xlabel('Value')
axes[0, 1].set_ylabel('Frequency')

# Min-Max
sns.histplot(expr_minmax_df.values.flatten(), bins=100, ax=axes[0, 2], kde=True)
axes[0, 2].set_title('Min-Max Scaling')
axes[0, 2].set_xlabel('Value')
axes[0, 2].set_ylabel('Frequency')

# Robust
sns.histplot(expr_robust_df.values.flatten(), bins=100, ax=axes[1, 0], kde=True)
axes[1, 0].set_title('Robust Scaling')
axes[1, 0].set_xlabel('Value')
axes[1, 0].set_ylabel('Frequency')

# Log
sns.histplot(expr_log_df.values.flatten(), bins=100, ax=axes[1, 1], kde=True)
axes[1, 1].set_title('Log Transformation')
axes[1, 1].set_xlabel('Value')
axes[1, 1].set_ylabel('Frequency')

# Quantile
sns.histplot(expr_quantile_normalized_df.values.flatten(), bins=100, ax=axes[1, 2], kde=True)
axes[1, 2].set_title('Quantile Normalization')
axes[1, 2].set_xlabel('Value')
axes[1, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
expr_normalized = expr_quantile_normalized_df

In [ ]:
# Save to CSV file
csv_file_path = 'data/expr_normalized.csv'
expr_normalized.to_csv(csv_file_path)

# Joining Omics datasets

In [ ]:
# Import annotated methylation from the R analysis
methylation_agg = pd.read_csv("data\methylation_agg.csv", index_col=0)
methylation_agg

In [ ]:
# Check for common indices 
common_indices = expr_normalized.index.intersection(mutations_imputed.index).intersection(methylation_agg.index)
common_indices_count = len(common_indices)

print(f"Common row indices between expr, mutations, and methylation: {common_indices_count}")
print(f"Common row indices: {common_indices.tolist()}")

In [ ]:
# Indexing meth, expr and mut data
methylation_agg.index = [f"{idx}_meth" for idx in methylation_agg.index]
expr_normalized.index = [f"{idx}_expr" for idx in expr_normalized.index]
mutations_imputed.index = [f"{idx}_mut" for idx in mutations_imputed.index]

In [ ]:
# Join normalized omics data 
multiomics_data = covariates.join([df.T for df in [drugs_normalized, methylation_agg, mutations_imputed, expr_normalized]], how='left')
multiomics_data

In [ ]:
# Check for missing values 
print(f'Total number of missing values in multiomics_data: {multiomics_data.isnull().sum().sum()}\n')

In [ ]:
# Drop Diagnosis
multiomics_data.drop(columns=['Diagnosis'], inplace=True)

In [ ]:
# Convert gender to numeric labels
multiomics_data['Gender'], unique_genders = pd.factorize(multiomics_data['Gender'])

In [ ]:
# Check gender codes
gender_mapping = {index: gender for index, gender in enumerate(unique_genders)}
print("Gender mapping:", gender_mapping)

In [ ]:
# Save to CSV file
csv_file_path = 'data/multiomics_data.csv'
multiomics_data.to_csv(csv_file_path)

In [ ]:
# PCA & t-SNE of multiomics data
pca = PCA(n_components=200)
pca_result = pca.fit_transform(multiomics_data.select_dtypes(include=[float, int]))

# Get explained variance ratio
explained_variance = pca.explained_variance_ratio_

# Perform t-SNE with a fixed random seed
tsne = TSNE(n_components=2, perplexity=30, n_iter=300, random_state=38)
tsne_result = tsne.fit_transform(multiomics_data.select_dtypes(include=[float, int]))

# Set up the plots side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Create numeric labels from the 'Gender' column
labels = multiomics_data['Gender']

# Plot PCA results
scatter_pca = axes[0].scatter(pca_result[:,0], pca_result[:,1], c=labels, cmap='viridis', s=50, edgecolor='k')
axes[0].set_title('PCA of Merged Data')
axes[0].set_xlabel(f'PCA Component 1 ({explained_variance[0]*100:.2f}% variance)')
axes[0].set_ylabel(f'PCA Component 2 ({explained_variance[1]*100:.2f}% variance)')
axes[0].grid(True)
colorbar_pca = fig.colorbar(scatter_pca, ax=axes[0])
colorbar_pca.set_label('Gender')

# Plot t-SNE results
scatter_tsne = axes[1].scatter(tsne_result[:,0], tsne_result[:,1], c=labels, cmap='viridis', s=50, edgecolor='k')
axes[1].set_title('t-SNE of Merged Data')
axes[1].set_xlabel('t-SNE Component 1')
axes[1].set_ylabel('t-SNE Component 2')
axes[1].grid(True)
colorbar_tsne = fig.colorbar(scatter_tsne, ax=axes[1])
colorbar_tsne.set_label('Gender')

# Enhance overall layout
plt.tight_layout()
plt.show()

# Feature Selection

## Correlation

In [ ]:
# Correlation Analysis
correlation_matrix = multiomics_data.corr()
gender_correlation = correlation_matrix['Gender'].sort_values(ascending=False)

# Top 2000 features 
top_features_corr = gender_correlation.head(2000).index.tolist()

In [ ]:
# Heatmap for selected top features
top_features = gender_correlation.head(2000)

sns.set(style="whitegrid")

plt.figure(figsize=(22, 12))
sns.heatmap(multiomics_data[top_features.index[:25]].corr().round(2), 
            annot=True, fmt='.2f', cmap='coolwarm', linewidths=.5, 
            mask=np.triu(np.ones_like(multiomics_data[top_features.index[:25]].corr(), dtype=bool), k=1),
            cbar_kws={"shrink": 0.8, "label": "Correlation Coefficient"}, annot_kws={"size": 10})

plt.title('Correlation Matrix of Top 25 Features with Gender', fontsize=20, weight='bold', pad=20)
plt.xticks(fontsize=12, rotation=45, ha="right", weight='bold')
plt.yticks(fontsize=12, weight='bold')
plt.tight_layout()
plt.show()

## Variance 

In [ ]:
# Variance 
X = multiomics_data.drop(columns=['Gender'])
y = multiomics_data['Gender']

selector = VarianceThreshold(threshold=0.1)
X_high_variance = selector.fit_transform(X)
top_features_var = X.columns[selector.get_support()]

print(f"Number of features before: {X.shape[1]}")
print(f"Number of features after: {X_high_variance.shape[1]}")

## Feature Importance

In [ ]:
# Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=15)
rf.fit(X, y)

feature_importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)

top_features_rf = feature_importances.head(2000).index.tolist()

## Statistical Test

In [ ]:
# SelectKBest
selector = SelectKBest(score_func=f_classif, k=2000)
selector.fit(X, y)
top_features_anova = X.columns[selector.get_support()].tolist()

## Intersect features 

In [ ]:
# Intersect features
all_features = list(top_features_var) + list(top_features_rf) + list(top_features_anova) + list(top_features_corr)
unique_features = list(set(all_features))
len(unique_features)

In [ ]:
# Create new df with intersected features
subset_columns = list(set(top_features_var) | set(top_features_rf) | set(top_features_anova) | set(top_features_corr))
subset_multiomics_data = multiomics_data[subset_columns]
subset_multiomics_data

## Multivariate Feature Selection

### Lasso

In [ ]:
# Lasso
X = subset_multiomics_data.drop(columns=['Gender'])  
y = subset_multiomics_data['Gender']

lasso = Lasso(alpha=0.01)
lasso.fit(X, y)
model = SelectFromModel(lasso, prefit=True)
selected_features_multivar = X.columns[(model.get_support())]

In [ ]:
# Create new df with Lasso features
lasso_data = subset_multiomics_data[selected_features_multivar.tolist() + ["Gender"]]
lasso_data

### SelectKBest

In [ ]:
# Check for constant features and `inf` values in the data
constant_features = [col for col in X.columns if X[col].nunique() == 1]
columns_with_inf = X.columns[np.isinf(X).any()]

print("Constant Features (same value for all samples):", constant_features if constant_features else "None")
print(f"Number of `inf` values in the dataset: {np.isinf(X).sum().sum()}")
print(f"Columns with `inf` values: {columns_with_inf.tolist()}")

In [ ]:
# Univariate Feature Selection
bestfeatures = SelectKBest(score_func=f_classif, k=2000)  
fit = bestfeatures.fit(X, y)
df_scores = pd.DataFrame(fit.scores_)
df_columns = pd.DataFrame(X.columns)

featureScores = pd.concat([df_columns, df_scores], axis=1)
featureScores.columns = ['Feature', 'Score']
selected_features = featureScores.nlargest(2000, 'Score')['Feature']

# Combine features with Lasso
combined_selected_features = list(set(selected_features) | set(selected_features_multivar))
X_selected = X.loc[:, combined_selected_features]
X_selected

In [ ]:
# StratifiedKFold
kf = StratifiedKFold(n_splits=5)
selected_features_per_fold = []

for train_index, test_index in kf.split(X_selected, y):
    X_train, X_test = X_selected.iloc[train_index], X_selected.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    selector = SelectKBest(score_func=f_classif, k=50) 
    X_train_selected = selector.fit_transform(X_train, y_train)

    selected_feature_names = X_train.columns[selector.get_support()]
    selected_features_per_fold.append(selected_feature_names)

In [ ]:
# Combine all selected features from all folds 
all_selected_features = set()
for features in selected_features_per_fold:
    all_selected_features.update(features)

filtered_multiomics_data = multiomics_data[list(all_selected_features) + ['Gender']]
filtered_multiomics_data

# Feature Engineering

## Methylation & Expression

In [ ]:
# Load methylation correlation data from R
gene_correlation = [gene for gene in pd.read_csv("data/gene_correlation.csv", header=None).iloc[:, 0] if gene != "x"]

In [ ]:
# Check for the presence of genes already in the data
suffixes = ['_expr', '_mut', '_meth']
present_columns = [gene + suffix for gene in gene_correlation for suffix in suffixes if gene + suffix in filtered_multiomics_data.columns]

# Print the count of present columns
print(f"Number of columns present: {len(present_columns)}")

In [ ]:
# Calculate the ratio between methylation and expression
for gene in gene_correlation:
    meth_col, expr_col = gene + '_meth', gene + '_expr'
    
    if {meth_col, expr_col}.issubset(multiomics_data.columns):
        multiomics_data[gene + '_corr'] = multiomics_data[meth_col] / multiomics_data[expr_col]
    else:
        print(f"Columns for {gene} not found in the data.")

In [ ]:
# Add _corr data to the data
corr_columns = [col for col in multiomics_data.columns if col.endswith('_corr')]
subset_multiomics_data_corr = multiomics_data[corr_columns]

In [ ]:
# Join _corr data to the filtered_multiomics_data
joined_data = filtered_multiomics_data.join(subset_multiomics_data_corr, how='left')
joined_data

In [ ]:
# Define threshold for extremely large values
extremely_large_value_threshold = 1e6

# Count NaN, infinity, and extremely large values
nan_count = joined_data.isna().sum().sum()
inf_count = np.isinf(joined_data).sum().sum()
neg_inf_count = np.isneginf(joined_data).sum().sum()
extremely_large_count = (joined_data > extremely_large_value_threshold).sum().sum()

# Print the results
print(f"Number of NaN values: {nan_count}")
print(f"Number of positive and negative infinity values: {inf_count}")
print(f"Number of negative infinity values: {neg_inf_count}")
print(f"Number of extremely large values (> {extremely_large_value_threshold}): {extremely_large_count}")

In [ ]:
# Filter rows and columns with +/- inf
filtered_inf_rows = joined_data[np.isinf(joined_data).any(axis=1)].loc[:, np.isinf(joined_data).any(axis=0)]
filtered_inf_rows

In [ ]:
# Replace inf value
inf_value_mask = (joined_data['FGL2_corr'] == -np.inf)
joined_data.loc[inf_value_mask, 'FGL2_corr'] = 1e+5

In [ ]:
# Correlation of all variables with the gender
correlations = joined_data.corr()['Gender'].drop('Gender')
sorted_correlations = correlations.abs().sort_values(ascending=False)

In [ ]:
# Significant correlations
significance_results = {col: pearsonr(joined_data[col], joined_data['Gender']) for col in sorted_correlations.index}
significance_df = pd.DataFrame(significance_results, index=['Correlation', 'p-value']).T
significance_df['adjusted p-value'] = multipletests(significance_df['p-value'], method='fdr_bh')[1]
significant_correlations = significance_df[significance_df['adjusted p-value'] < 0.05]
significant_correlations

In [ ]:
# Subset significant_correlations to the joined_data
significant_columns = significant_correlations.index.tolist() + ['Gender']
joined_data_2 = joined_data[significant_columns]
joined_data_2

## Drugs Clustering

In [ ]:
# Subset data with drugs and Gender
subset_multiomics_data = multiomics_data[['Gender']].join(multiomics_data.filter(regex='^D_'))
subset_multiomics_data

In [ ]:
# K-Means Clustering
kmeans = KMeans(n_clusters=2, random_state=42)  
clusters = kmeans.fit_predict(subset_multiomics_data)

# Add cluster labels as a new feature
drugs_clustered_df = subset_multiomics_data.copy()
drugs_clustered_df['Drug_Cluster'] = clusters
drugs_clustered_df

In [ ]:
# Add drugs_clustered data 
joined_data_3 = joined_data_2.join(drugs_clustered_df[['Drug_Cluster']], how='left')
joined_data_3

In [ ]:
# Correlation of all variables with gender
correlations = joined_data_3.corr()['Gender'].drop('Gender')
sorted_correlations = correlations.abs().sort_values(ascending=False)

In [ ]:
# Adj. p vals
significance_results = {col: pearsonr(joined_data_3[col], joined_data_3['Gender']) for col in sorted_correlations.index}
significance_df = pd.DataFrame(significance_results, index=['Correlation', 'p-value']).T
significance_df['adjusted p-value'] = multipletests(significance_df['p-value'], method='fdr_bh')[1]
significant_correlations = significance_df[significance_df['adjusted p-value'] < 0.05]
significant_correlations

## VIF

In [ ]:
# VIF
X = joined_data_3.drop(columns=['Gender'])

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
vif_data

In [ ]:
# Filter VIF with threshold
vif_threshold = 10
low_vif_vars = vif_data[vif_data["VIF"] < vif_threshold]["Variable"]

joined_data_3_low_vif = joined_data_3[low_vif_vars].copy()
joined_data_3_low_vif.loc[:, 'Gender'] = joined_data_3['Gender']
joined_data_3_low_vif

# Models

In [ ]:
# Function to evaluate models
def evaluate_model(model):
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    train_accuracy = round(model.score(X_train, y_train), 5)
    test_accuracy = round(model.score(X_test, y_test), 6)

    eval_metrics = pd.DataFrame({
        'Metric': ['Train Accuracy', 'Test Accuracy', 'Mean Absolute Error', 'Mean Squared Error', 'Root Mean Squared Error', 'R2 Score'],
        'Value': [
            train_accuracy,
            test_accuracy,
            metrics.mean_absolute_error(y_test, y_pred),
            metrics.mean_squared_error(y_test, y_pred),
            np.sqrt(metrics.mean_squared_error(y_test, y_pred)),
            metrics.r2_score(y_test, y_pred)]
    })
    
    return eval_metrics

In [ ]:
# Splitting into features(X) and target(y)
X = joined_data_3_low_vif.drop(columns=['Gender'])  # Drop target variable
y = joined_data_3_low_vif['Gender']

In [ ]:
# Splitting into train and test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 15)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
# Linear Regression Model
lr_model = LinearRegression()
lr_model_eval = evaluate_model(lr_model)
lr_model_eval

In [ ]:
# Ridge Regresion Model
rr_model = Ridge(random_state=15)
rr_model_eval = evaluate_model(rr_model)
rr_model_eval

In [ ]:
# Decision tree Model
dt_model = DecisionTreeRegressor(min_samples_split = 15, min_samples_leaf = 15,random_state = 15)
dt_model_eval = evaluate_model(dt_model)
dt_model_eval

In [ ]:
# Random Forest Model
rf_model = RandomForestRegressor(n_estimators = 100, max_depth = 15, min_samples_split = 5, max_features = 'auto', random_state = 15) # 
rf_model_eval = evaluate_model(rf_model)
rf_model_eval

In [ ]:
# XGboost Model
xgb_model = XGBRegressor(max_depth = 2, learning_rate = 0.04, n_estimators = 200, min_child_weight = 6, subsample = 0.7, colsample_bytree = 0.7, gamma = 0.05, random_state = 15) # early_stopping_rounds to 10 or 20
xgb_model_eval = evaluate_model(xgb_model)
xgb_model_eval

In [ ]:
# Plot important features
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': xgb_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

plot_importance(xgb_model, max_num_features=10)
plt.show()

In [ ]:
# Model comparison
lr_model_eval['Model'] = 'lr'
rr_model_eval['Model'] = 'rr'
dt_model_eval['Model'] = 'dt'
rf_model_eval['Model'] = 'rf'
xgb_model_eval['Model'] = 'xgb'

model_comp = pd.concat([lr_model_eval, rr_model_eval, dt_model_eval, rf_model_eval, xgb_model_eval])
model_comp = model_comp.pivot(index='Model', columns='Metric', values='Value')
model_comp = model_comp.sort_values(by='Test Accuracy', ascending=False)
model_comp

In [ ]:
# Save the xgb model
joblib.dump(xgb_model, 'data/xgb_model.pkl')